In [2]:
import pandas
import matplotlib
% matplotlib inline

In [3]:
import os
os.chdir("../../Documents/Portfolio/Education project/")
os.listdir("./")

['adm2014.csv',
 'CollegeScorecardDataDictionary-09-08-2015.csv',
 'CollegeScorecard_Raw_Data (2)',
 'CollegeScorecard_Raw_Data (2).zip',
 'g20145ak.txt',
 'Integrated_Postsecondary_Education_Data_System_20132014.csv',
 'merged_2011_PP.csv',
 'merged_2012_PP.csv',
 'merged_2013_PP.csv',
 'Objective Hypothesis.docx',
 'US Zip Codes from 2013 Government Data']

In [4]:
data_2011 = pandas.read_csv("merged_2011_PP.csv")
data_2012 = pandas.read_csv("merged_2012_PP.csv")
data_2013 = pandas.read_csv("merged_2013_PP.csv")

C:\Users\Marissa\AppData\Local\Enthought\Canopy\User\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (1517,1532,1575) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Marissa\AppData\Local\Enthought\Canopy\User\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (9,1561,1575,1725,1726,1727,1728) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Create map to decode variable names

In [5]:
dictionary_data = pandas.read_csv("CollegeScorecardDataDictionary-09-08-2015.csv")
temp_map_variables = dictionary_data.loc[:,["NAME OF DATA ELEMENT","VARIABLE NAME"] ].apply(lambda x: {x[1]: x[0]}, axis = 1 )
map_variables = {}
for element in temp_map_variables:
    col_name = element.keys() 
    #print col_name
    map_variables.update(element)

### Backfill values for categories only available in 2013

Some of the most recent data is only in 2013 data while the completion rates, repayment rates, etc.. are only available to 2012. From opendata.stackexchange.com:
Most of the data on student outcomes (e.g., completion rates, repayment rates, and debt measures) are not available past the 2013-14 year at this time. For some elements, like the earnings data, the most recent data available describe cohorts measured in the 2011 and 2012 tax years. However, a few elements—including header data like institutional name and address, Minority Serving Institution status, currently operating status, and HCM2 status—are more recent. These data are, for the purposes of convenience, also included in the 2013 file (which comprises both 2013 and other more recent data). In our next annual update, we will add another file, which will include 2014-15 data for may outcomes, as well as more recent data related to the institution. We recognize this can be an unwieldy way to work with the data, so we will continue to explore ways to make this information clearer and easier to access.
This means that we need to merge fields like Minoring Serving Institution status, currently operating status, and HCM2 status from the 2012 data with the 2013 data.


In [6]:
data = data_2011
data_count = data.count() 
print (data_count.values < 100).sum()
drop_2011 = []
for i, name in enumerate(data.columns):
    if data_count[name] < 100:
        #print "%5i %35s %10i" %(i, name, data_count[[i]])
        #try:
            #print "      %90s" %map_variables[name]
        #except:
            #print "None"
        drop_2011.append(name)
        

161


In [7]:
data = data_2013
data_count = data.count() 
print (data_count.values < 100).sum()
drop_2013 = []
for i, name in enumerate(data.columns):
    if data_count[name] < 100:
        #print "%5i %35s %10i" %(i, name, data_count[[i]])
        #try:
        #    print "      %90s" %map_variables[name]
        #except:
        #    print "None"
        drop_2013.append(name)



1174


In [8]:
print "2011:" , len(drop_2011)
print "2013:" , len(drop_2013)
def intersect(a,b):
    return list(set(a) & set(b))
intersect_2011_2013 = intersect(drop_2011, drop_2013)
print "Intersection 2011-2013", len(intersect_2011_2013)

print "\n\nWhat variables are in the 2013 data and not in the 2011 data"
for i, n in enumerate(sorted(set(drop_2011) - set(drop_2013))):
    print i, n, map_variables.get(n, "None")
fill_2013 = sorted(set(drop_2011) - set(drop_2013))

2011: 161
2013: 1174
Intersection 2011-2013 98


What variables are in the 2013 data and not in the 2011 data
0 AANAPII Flag for Asian American Native American Pacific Islander-serving institution
1 ANNHI Flag for Alaska Native Native Hawaiian serving institution
2 AccredAgency Accreditor for institution
3 C150_4_POOLED Completion rate for first-time, full-time students at four-year institutions (150% of expected time to completion/6 years), pooled for two year rolling averages
4 C150_4_POOLED_SUPP 150% completion rate for four-year institutions, pooled in two-year rolling averages and suppressed for small n size.  For four year school, students are considered to have graduated "on time" if they graduate within 6 years.
5 C150_L4_POOLED Completion rate for first-time, full-time students at less-than-four-year institutions (150% of expected time to completion), pooled for two year rolling averages
6 C150_L4_POOLED_SUPP 150% completion rate for less-than-four-year institutions, pooled in

In [9]:
## Variables to get from 2013 data
Specialized_mission = ["AANAPII", "ANNHI", "HBCU", "HSI", "MENONLY", "NANTI", "PBI", "TRIBAL", "WOMENONLY"]
Base_descriptors = ["CCBASIC", "CCSIZSET", "CCUGPROF", "CURROPER", "LATITUDE", "LONGITUDE", "LOCALE"]
Pooled_Completion = [name for name in fill_2013 if name[0:4] in ["C150", "C200", "D150", "D200", "pool"]]

Repayment = [name for name in fill_2013 if "RPY_7YR" in name]
print Pooled_Completion
print Repayment

print [name for name in fill_2013 if name not in (Specialized_mission + Base_descriptors + Pooled_Completion + Repayment)]

['C150_4_POOLED', 'C150_4_POOLED_SUPP', 'C150_L4_POOLED', 'C150_L4_POOLED_SUPP', 'C200_4_POOLED', 'C200_4_POOLED_SUPP', 'C200_L4_POOLED', 'C200_L4_POOLED_SUPP', 'D150_4_POOLED', 'D150_L4_POOLED', 'D200_4_POOLED', 'D200_L4_POOLED', 'poolyrs', 'poolyrs200']
['COMPL_RPY_7YR_N', 'COMPL_RPY_7YR_RT', 'DEP_RPY_7YR_N', 'DEP_RPY_7YR_RT', 'FEMALE_RPY_7YR_N', 'FEMALE_RPY_7YR_RT', 'FIRSTGEN_RPY_7YR_N', 'FIRSTGEN_RPY_7YR_RT', 'HI_INC_RPY_7YR_N', 'HI_INC_RPY_7YR_RT', 'IND_RPY_7YR_N', 'IND_RPY_7YR_RT', 'LO_INC_RPY_7YR_N', 'LO_INC_RPY_7YR_RT', 'MALE_RPY_7YR_N', 'MALE_RPY_7YR_RT', 'MD_INC_RPY_7YR_N', 'MD_INC_RPY_7YR_RT', 'NONCOM_RPY_7YR_N', 'NONCOM_RPY_7YR_RT', 'NOPELL_RPY_7YR_N', 'NOPELL_RPY_7YR_RT', 'NOTFIRSTGEN_RPY_7YR_N', 'NOTFIRSTGEN_RPY_7YR_RT', 'PELL_RPY_7YR_N', 'PELL_RPY_7YR_RT', 'RPY_7YR_N', 'RPY_7YR_RT']
['AccredAgency', 'HCM2', 'INSTURL', 'NPCURL', 'RELAFFIL']


### Combine these variables with the 2011 data


In [10]:
print data_2011["AANAPII"].count()

data_2011[(Specialized_mission + Base_descriptors + Pooled_Completion + Repayment)] = data_2013[(Specialized_mission + Base_descriptors + Pooled_Completion + Repayment)]

print data_2011["AANAPII"].count()
print data_2013["AANAPII"].count()

0
7383
7383


### Trim out the columns with a lot of nulls


In [11]:
data = data_2011
data_count = data.count() 
print (data_count.values < 1000).sum()
data_trim = data.copy()
errors = []
drop_cols = []
for i, name in enumerate(data.columns):
    if data_count[name] < 1000:
        print "%5i %35s %10i" %(i, name, data_count[[i]])
        try:
            print "      %90s" %map_variables[name]
        except:
            print "None"
        try:
            data_trim = data_trim.drop([name], axis = 1)
            drop_cols.append(name)
        except:
            errors.append(name)

114
    7                        AccredAgency          0
                                                                      Accreditor for institution
    8                             INSTURL          0
                                                                  URL for institution's homepage
    9                              NPCURL          0
                                                      URL for institution's net price calculator
   11                                HCM2          0
                 Schools that are on Heightened Cash Monitoring 2 by the Department of Education
   20                             locale2          0
                                                           Degree of urbanization of institution
   35                            RELAFFIL          0
                                                         Religous affiliation of the institution
   42                             SATWR25        748
                                      25th 

In [12]:
data_count = data_trim.count() 
print (data_count.values < 4000).sum()
for i, name in enumerate(data_trim.columns):
    if data_count.values[[i]] < 4000:
        print i, name, data_count[name]
        print "%5i %35s %10i" %(i, name, data_count[[i]])
        try:
            print "      %90s" %map_variables[name]
        except:
            print "None"

77
19 CCUGPROF 3559
   19                            CCUGPROF       3559
                                                Carnegie Classification -- undergraduate profile
20 CCSIZSET 3576
   20                            CCSIZSET       3576
                                                     Carnegie Classification -- size and setting
30 ADM_RATE 2347
   30                            ADM_RATE       2347
                                                                                  Admission rate
31 ADM_RATE_ALL 2638
   31                        ADM_RATE_ALL       2638
                                 Admission rate for all campuses rolled up to the 6-digit OPE ID
32 SATVR25 1267
   32                             SATVR25       1267
                             25th percentile of SAT scores at the institution (critical reading)
33 SATVR75 1267
   33                             SATVR75       1267
                             75th percentile of SAT scores at the institution (critical re

In [13]:
data_trim.columns

Index([u'UNITID', u'OPEID', u'opeid6', u'INSTNM', u'CITY', u'STABBR', u'ZIP',
       u'sch_deg', u'main', u'NUMBRANCH',
       ...
       u'PELL_RPY_3YR_RT_SUPP', u'NOPELL_RPY_3YR_RT_SUPP',
       u'FEMALE_RPY_3YR_RT_SUPP', u'MALE_RPY_3YR_RT_SUPP',
       u'FIRSTGEN_RPY_3YR_RT_SUPP', u'NOTFIRSTGEN_RPY_3YR_RT_SUPP',
       u'C150_L4_POOLED_SUPP', u'C150_4_POOLED_SUPP', u'C200_L4_POOLED_SUPP',
       u'C200_4_POOLED_SUPP'],
      dtype='object', length=1615)

### Filter academic institutions

Having the SAT and ACT scores is important and seems an efficient filter to the institutions with more complete data. So does the question of whether a school is currently operational or not.

In [14]:
data_trim.groupby("CURROPER").count()

,UNITID,OPEID,opeid6,INSTNM,CITY,STABBR,ZIP,sch_deg,main,NUMBRANCH,...,PELL_RPY_3YR_RT_SUPP,NOPELL_RPY_3YR_RT_SUPP,FEMALE_RPY_3YR_RT_SUPP,MALE_RPY_3YR_RT_SUPP,FIRSTGEN_RPY_3YR_RT_SUPP,NOTFIRSTGEN_RPY_3YR_RT_SUPP,C150_L4_POOLED_SUPP,C150_4_POOLED_SUPP,C200_L4_POOLED_SUPP,C200_4_POOLED_SUPP
CURROPER,,,,,,,,,,,,,,,,,,,,,
0,470,470,470,470,470,470,470,463,470,470,...,384,384,384,384,384,384,307,70,284,57
1,7205,7205,7205,7205,7205,7205,7205,7117,7205,7205,...,6119,6119,6119,6119,6119,6119,3711,2402,3510,2103


In [15]:
import numpy as np
# Check that there is a value in either SAT_AVG or ACTCMMID
trim_2 = data_trim[data_trim.apply(lambda x: np.isfinite(x["SAT_AVG"]) or np.isfinite(x["ACTCMMID"]), axis=1)]
# Check that the school is still in operation
trim_3 = trim_2[trim_2["CURROPER"] == 1]
print "Total institutions", trim_3["INSTNM"].count()
for name in sorted(trim_3.columns):
    print trim_3[name].count(), name, map_variables.get(name, "None")


Total institutions 1361
1361 AANAPII Flag for Asian American Native American Pacific Islander-serving institution
1278 ACTCM25 25th percentile of the ACT cumulative score
1278 ACTCM75 75th percentile of the ACT cumulative score
1278 ACTCMMID Midpoint of the ACT cumulative score
1086 ACTEN25 25th percentile of the ACT English score
1086 ACTEN75 75th percentile of the ACT English score
1086 ACTENMID Midpoint of the ACT English score
1085 ACTMT25 25th percentile of the ACT math score
1085 ACTMT75 75th percentile of the ACT math score
1085 ACTMTMID Midpoint of the ACT math score
1361 ADM_RATE Admission rate
1361 ADM_RATE_ALL Admission rate for all campuses rolled up to the 6-digit OPE ID
1361 ANNHI Flag for Alaska Native Native Hawaiian serving institution
1361 APPL_SCH_N Number of students in the FAFSA applications cohort
1361 APPL_SCH_PCT_GE2 Number of applications is greater than or equal to 2
1361 APPL_SCH_PCT_GE3 Number of applications is greater than or equal to 3
1361 APPL_SCH_PCT_G

In [16]:
trim_3.shape


(1361, 1615)

### Categorize the remaining variables

In [17]:


## Variable categories:

#### General
Special_mission = ["AANAPII", "ANNHI", "HBCU", "HSI", "MENONLY", "NANTI", "PBI", "TRIBAL", "WOMENONLY"]
print "Special_mission: ", Special_mission

Carnegie_info = ["CCBASIC", "CCSIZSET", "CCUGPROF"]
print "\nCarnegie classification:", Carnegie_info
School_info = ["CONTROL", "HIGHDEG", "PREDEG"]
print "\nSchool_info:",School_info
Dependence = [name for name in trim_3.columns if name[0:6] in ["DEP_ST", ]]
print "\nDepencence", Dependence
Location = ["CITY", "LATTITUDE", "LONGITUDE", "LOCALE", "STABBR", "region"]
print "\nLocation", Location
# Student composition
Parent_ed = [name for name in trim_3.columns if name[0:6] in ["PAR_ED", ]]
Older_students = [name for name in trim_3.columns if name[0:4] == "UG25"]

Program_vars = [name for name in trim_3.columns if name[0:3] == "CIP"]
Program_percent = [name for name in trim_3.columns if name[0:4] in ["PCIP", ]]

Full_vs_part = [name for name in trim_3.columns if name[0:3] in ["PFT","PPT" ]]

Test_scores = [name for name in trim_3.columns if name[0:3] in ["ACT", "SAT"]]
print "Test scores: ", Test_scores


###### Academic trajectory
Undergraduate_enrollment = [name for name in trim_3.columns if name[0:4] == "UDGS"]
# conditioned on race
Admissions_rate = [name for name in trim_3.columns if name[0:3] == "ADM"]
print "Admissions: ", Admissions_rate
Still_enrolled = [name for name in trim_3.columns if "ENRL_ORIG" in name]
# conditioned on Dependent, gender, firstgen, income level, loan received, pell received
Transfer_rate = [name for name in trim_3.columns if "TRANS_" in name]
# Conditioned on dependent, gender
Unknown_status = [name for name in trim_3.columns if "UNKN_ORIG" in name]
# Conditioned on dependent, gender
Withdrawn = [name for name in trim_3.columns if "WDRAW_ORIG" in name]
Retention_rate = [name for name in trim_3.columns if name[0:3] == "RET"]

Completion_rate = [name for name in trim_3.columns if (
        name[0:4] in ["C150", "C200", "D150", "D200"]) or (
        "COMP_ORIG" in name) or (("_YR" in name) and ("_N" in name))]
print "Completion: ", Completion_rate
# C150 Conditioned on race. gender, and dependent status and pooled for 2 yr rolling avg
    # Races: American Indian/Alaska Native, Asian, black, Hispanic, unknown, white
# Need to merge the C150_4 variables and the C150_L4 variables


#### Financial impact
Cost = [name for name in trim_3.columns if name[0:4] in ["COST", "TUIT"]]
Avg_Price = [name for name in trim_3.columns if (name[0:3] == "NPT") and (name not in ["NPT4_PRIV","NPT4_PUB"])]
Title_4 = [name for name in trim_3.columns if ((name[0:3] == "NUM") and name != "NUMBRANCH") or (name in ["NPT4_PRIV","NPT4_PUB"])]
# Split public/private and conditioned on income    
Debt = ([name for name in trim_3.columns if name[0:4] in ["CUML", "DEBT"]] +
        [name for name in trim_3.columns if name[0:9] in ["GRAD_DEBT"]])
# DEBT conditioned on dependent, gender
Income = [name for name in trim_3.columns if "_INC_" in name]
# conditioned on Dependent
Death = [name for name in trim_3.columns if "DEATH" in name]
# conditioned on Dependent, gender
Repayment_rate = [name for name in trim_3.columns if ("RPY_" in name) or (name[0:5] == "REPAY")]
# conditioned on Dependent, gender, completion
print "Repayment: ", Repayment_rate
Default_rate = ["CDR2", "CDR3"]
Employment_status = [name for name in trim_3.columns if name[0:5] == "count"]
Earnings = [name for name in trim_3.columns if name[0:3] in ["gt", "md", "mn", "pct", "sd"]]
            
###### Financial Aid
FAFSA_applications = [name for name in trim_3.columns if name[0:4] == "APPL"]
print "FAFSA_applications: ", FAFSA_applications

Aided_percentiles = [name for name in trim_3.columns if name[0:3] in ["INC", "PCT"]]
print "Aided_percentiles: ", Aided_percentiles



Special_mission:  ['AANAPII', 'ANNHI', 'HBCU', 'HSI', 'MENONLY', 'NANTI', 'PBI', 'TRIBAL', 'WOMENONLY']

Carnegie classification: ['CCBASIC', 'CCSIZSET', 'CCUGPROF']

School_info: ['CONTROL', 'HIGHDEG', 'PREDEG']

Depencence ['DEP_STAT_PCT_IND', 'DEP_STAT_N']

Location ['CITY', 'LATTITUDE', 'LONGITUDE', 'LOCALE', 'STABBR', 'region']
Test scores:  ['SATVR25', 'SATVR75', 'SATMT25', 'SATMT75', 'SATVRMID', 'SATMTMID', 'ACTCM25', 'ACTCM75', 'ACTEN25', 'ACTEN75', 'ACTMT25', 'ACTMT75', 'ACTCMMID', 'ACTENMID', 'ACTMTMID', 'SAT_AVG', 'SAT_AVG_ALL']
Admissions:  ['ADM_RATE', 'ADM_RATE_ALL']
Completion:  ['C150_4', 'C150_L4', 'C150_4_POOLED', 'C150_L4_POOLED', 'D150_4', 'D150_L4', 'D150_4_POOLED', 'D150_L4_POOLED', 'C150_4_WHITE', 'C150_4_BLACK', 'C150_4_HISP', 'C150_4_ASIAN', 'C150_4_AIAN', 'C150_4_NRA', 'C150_4_UNKN', 'C150_L4_WHITE', 'C150_L4_BLACK', 'C150_L4_HISP', 'C150_L4_ASIAN', 'C150_L4_AIAN', 'C150_L4_UNKN', 'C200_4', 'C200_L4', 'D200_4', 'D200_L4', 'C200_4_POOLED', 'C200_L4_POOLED', 'D2

In [18]:
# Program names
Program_vars = [name for name in trim_3.columns if name[0:3] == "CIP"]
Program_names = []
for var in Program_vars:
    program = map_variables[var].split(" in ")[1].strip()
    Program_names.append((program, var[3:5]))
Program_names = sorted(list(set(Program_names)), key = lambda x: x[1])
print len(Program_names)
for program_tuple in Program_names:
    print program_tuple

38
('Agriculture, Agriculture Operations, And Related Sciences.', '01')
('Natural Resources And Conservation.', '03')
('Architecture And Related Services.', '04')
('Area, Ethnic, Cultural, Gender, And Group Studies.', '05')
('Communication, Journalism, And Related Programs.', '09')
('Communications Technologies/Technicians And Support Services.', '10')
('Computer And Information Sciences And Support Services.', '11')
('Personal And Culinary Services.', '12')
('Education.', '13')
('Engineering.', '14')
('Engineering Technologies And Engineering-Related Fields.', '15')
('Foreign Languages, Literatures, And Linguistics.', '16')
('Family And Consumer Sciences/Human Sciences.', '19')
('Legal Professions And Studies.', '22')
('English Language And Literature/Letters.', '23')
('Liberal Arts And Sciences, General Studies And Humanities.', '24')
('Library Science.', '25')
('Biological And Biomedical Sciences.', '26')
('Mathematics And Statistics.', '27')
('Military Technologies And Applied Scie

## Data clean-up steps
- Need to merge the C150_4 variables and the C150_L4 variables

- merge the columns that express values separately for public and private schools




In [25]:
public = [name for name in trim_3.columns if "_PUB" in name]
print public
private = [name for name in trim_3.columns if "_PRIV" in name]
print private

['NPT4_PUB', 'NPT41_PUB', 'NPT42_PUB', 'NPT43_PUB', 'NPT44_PUB', 'NPT45_PUB', 'NPT4_048_PUB', 'NPT4_3075_PUB', 'NPT4_75UP_PUB', 'NUM4_PUB', 'NUM41_PUB', 'NUM42_PUB', 'NUM43_PUB', 'NUM44_PUB', 'NUM45_PUB']
['NPT4_PRIV', 'NPT41_PRIV', 'NPT42_PRIV', 'NPT43_PRIV', 'NPT44_PRIV', 'NPT45_PRIV', 'NPT4_048_PRIV', 'NPT4_3075_PRIV', 'NPT4_75UP_PRIV', 'NUM4_PRIV', 'NUM41_PRIV', 'NUM42_PRIV', 'NUM43_PRIV', 'NUM44_PRIV', 'NUM45_PRIV']


In [32]:
# Merge columns
print trim_3.shape
for pub_name in public:
    print pub_name[:-3], map_variables[pub_name]
    priv_name = pub_name[:-3] + "PRIV"
    combined = pub_name[:-3] + "COMB"
    print "   Public", trim_3[pub_name].count()
    print "   Private", trim_3[priv_name].count()
    trim_3.loc[:,combined] = trim_3[pub_name].add(trim_3[priv_name] , fill_value = 0)
    print "   Combined:", trim_3[combined].count()
    map_variables[combined] = "Average net price for Title IV institutions"
    

(1361, 1630)
NPT4_ Average net price for Title IV institutions (public institutions)
   Public 516
   Private 841
   Combined: 1357
NPT41_ Average net price for $0-$30,000 family income (public institutions)
   Public 516
   Private 836
   Combined: 1352
NPT42_ Average net price for $30,001-$48,000 family income (public institutions)
   Public 514
   Private 831
   Combined: 1345
NPT43_ Average net price for $48,001-$75,000 family income (public institutions)
   Public 513
   Private 828
   Combined: 1341
NPT44_ Average net price for $75,001-$110,000 family income (public institutions)
   Public 508
   Private 817
   Combined: 1325
NPT45_ Average net price for $110,000+ family income (public institutions)
   Public 485
   Private 793
   Combined: 1278
NPT4_048_ Average net price for $0-$48,000 family income (public institutions)
   Public 516
   Private 838
   Combined: 1354
NPT4_3075_ Average net price for $30,001-$75,000 family income (public institutions)
   Public 516
   Private 83

In [34]:
trim_4 = trim_3.drop(public + private, axis = 1)

In [35]:
trim_4.shape

(1361, 1600)

In [36]:
trim_4.head()

,UNITID,OPEID,opeid6,INSTNM,CITY,STABBR,ZIP,sch_deg,main,NUMBRANCH,...,NPT45_COMB,NPT4_048_COMB,NPT4_3075_COMB,NPT4_75UP_COMB,NUM4_COMB,NUM41_COMB,NUM42_COMB,NUM43_COMB,NUM44_COMB,NUM45_COMB
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,3.0,1,1,...,13892.0,7944.0,9986.0,12422.0,644.0,381.0,118.0,82.0,41.0,22.0
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,3.0,1,1,...,19440.0,11547.0,14760.0,19339.0,907.0,288.0,162.0,177.0,144.0,136.0
3,100706,105500,1055,University of Alabama at Huntsville,Huntsville,AL,35899,3.0,1,1,...,12551.0,7397.0,10517.0,12219.0,303.0,102.0,48.0,53.0,48.0,52.0
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,3.0,1,1,...,9281.0,5098.0,6911.0,9709.0,639.0,460.0,103.0,39.0,26.0,11.0
5,100751,105100,1051,The University of Alabama,Tuscaloosa,AL,35487-0166,3.0,1,1,...,20429.0,14417.0,16911.0,19818.0,1373.0,498.0,247.0,226.0,241.0,161.0


In [43]:
trim_4.to_csv("Scorecard_v1.csv")
dict_file = open("Scorecard_dict.txt", "w")
for key, value in map_variables.iteritems():
    dict_file.write("%10s|%s\n" %(key, str(value).strip()))
dict_file.close()
